In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from tqdm import tqdm
from time import sleep

In [2]:
class Particle:

    def __init__(self,r0,v0,a0,t,m=1,radius=0.3,Id=0):

        self.dt = t[1] - t[0]

        self.r = r0
        self.v = v0
        self.a = a0

        self.R = np.zeros((len(t),len(r0)))
        self.V = np.zeros_like(self.R)
        self.A = np.zeros_like(self.R)

        self.radius = radius

    def Contact_Force(self, other):
        R1 = self.radius
        R2 = other.radius
        r1 = self.r
        r2 = other.r
        m = self.mass
        K = 100 / pow(m, 3)
        r_p = np.linalg.norm(r1 - r2)**3

        if r_p < R1 + R2:
            return K*r_p
        else:
            return 0

    def Evolution(self, i, other=None):

        self.SetPosition(i)
        self.SetVelocity(i)
        self.SetAcceleration(i)

        if other != None and self.Contact_Force(self, other) != 0:
            self.v += self.dt * self.Contact_Force(self, other)/self.mass
            self.r += self.dt * self.v
        else:
            self.v += self.dt * self.a
            self.r += self.dt * self.v

    def SetPosition(self,i):
        self.R[i] = self.r

    def GetPosition(self,scale=1):
        return self.R[::scale]

    def SetVelocity(self,i):
        self.V[i] = self.v

    def GetVelocity(self,scale=1):
        return self.V[::scale]

    def SetAcceleration(self,i):
        self.A[i] = self.a

    def GetAcceleration(self,scale=1):
        return self.A[::scale]

    def CheckLimits(self,Limits):

        for i in range(2):

            if self.r[i] + self.radius > Limits[i][1] and self.v[i] > 0.:
                self.v[i] = -1.0*self.v[i]
            if self.r[i] - self.radius < Limits[i][0] and self.v[i] < 0.:
                self.v[i] = -1.0*self.v[i]

In [3]:
def RunSimulation1(t,Wall):
    
    r0 = np.array([-15.,1.])
    v0 = np.array([10.,0.])
    a0 = np.array([0.,0.])
    
    
    p1 = Particle(r0,v0,a0,t)
    p2 = Particle(r0,v0,a0,t)
    
    Wall_ = Wall.copy()
    
    
    for it in tqdm(range(len(t)), desc='Running simulation', unit=' Steps'):
        sleep(0.0001)
        p1.Evolution(it, p2)
        p2.Evolution(it, p1)
        p1.CheckLimits(Wall_)
        p2.CheckLimits(Wall_)
    
    return p1

In [4]:
# Region
Limits = np.array([[-5.,5.],[-5.,5.]])
Limits

array([[-5.,  5.],
       [-5.,  5.]])

In [5]:
dt = 0.05
tmax = 10
t = np.arange(0.,tmax,dt)
Particles = RunSimulation1(t,Limits)

Running simulation: 100%|██████████| 200/200 [00:00<00:00, 5095.86 Steps/s]
